In [1]:
import numpy as np
import tensorflow as tf
import glob
import fastText as ft
import re
import os
config_gpu = tf.ConfigProto()
config_gpu.gpu_options.per_process_gpu_memory_fraction = 0.8
sess = tf.Session(config=config_gpu)

In [5]:
word = ft.load_model('wiki.vi.bin')

In [6]:
import shutil
shutil.rmtree('data_train')
os.mkdir('data_train')
tmp=dict()
for i in glob.glob('data/*.txt'):
    with open(i,'r') as f:
        tmp[i.split('/')[-1].split('.')[0]]=[j for j in f.read().splitlines() if len(j)!=0]

In [7]:
name=dict(zip(tmp.keys(),list(range(len(tmp.keys())))))
os.makedirs('data_train',exist_ok=True)
# for label in tmp.keys():
#     data=tmp[label]    
#     i=np.random.choice(len(data),len(data),replace=False)
#     iter_train=i[:9*len(data)//10]
#     iter_vaild=i[9*len(data)//10:]
#     for j in iter_train:        
#         sentences=re.split(r'(?<!\d)\.(?!\d)',data[j])
#         if sentences[-1]=='':
#             del(sentences[-1])        
#         for s in range(len(sentences)):
#             words_vecor=list()
#             for w in sentences[s].split(' '):
#                 w=re.compile('[",!?\[\]\(\)]-_=').sub('',w)
#                 words_vecor.append(word.get_word_vector(w))
#             sentences[s]=np.array(words_vecor)
#         data_train.append(sentences)
#         label_train.append(name[label])
#     for j in iter_vaild:        
#         sentences=re.split(r'(?<!\d)\.(?!\d)',data[j])
#         if sentences[-1]=='':
#             del(sentences[-1])        
#         for s in range(len(sentences)):
#             words_vecor=list()
#             for w in sentences[s].split(' '):
#                 w=re.compile('[",!?\[\]\(\)]-_=').sub('',w)
#                 words_vecor.append(word.get_word_vector(w))
#             sentences[s]=np.array(words_vecor)
#         data_vaild.append(sentences)
#         label_vaild.append(name[label])
d_train=list()
l_train=list()
d_vaild=list()
l_vaild=list()
for label in tmp.keys():
    data=tmp[label]    
    i=np.random.choice(len(data),len(data),replace=False)
    iter_train=i[:9*len(data)//10]
    iter_vaild=i[9*len(data)//10:]

    for j in iter_train:        
#         sentences=[s for s in re.split(r'(?<!\d)\.(?!\d)',data[j]) if s!='']
        words=re.sub(' +',' ',re.compile('[^\w0-9 ]').sub('',data[j])).split(' ')
        d_train.append(np.array([word.get_word_vector(i) for i in words]))
        l_train.append(name[label])

    for j in iter_vaild:        
        words=re.sub(' +',' ',re.compile('[^\w0-9 ]').sub('',data[j])).split(' ')
        d_vaild.append(np.array([word.get_word_vector(i) for i in words]))
        l_vaild.append(name[label])
np.save('data_train/train',[d_train,l_train])
np.save('data_train/vaild',[d_vaild,l_vaild])

In [2]:
data_train,label_train=np.load('data_train/train.npy')
data_vaild,label_vaild=np.load('data_train/vaild.npy')
len_train=np.argsort([i.shape[0] for i in data_train])
len_vaild=np.argsort([i.shape[0] for i in data_vaild])

In [3]:
# Training Parameters# Train 
starter_learning_rate = 0.001
max_epoch = 200
batch_size=50
num_class=len(glob.glob('data/*.txt'))

In [4]:
length=tf.placeholder(dtype=tf.int32,shape=[None])
global_step = tf.placeholder(tf.int32)
X =tf.placeholder(dtype=tf.float32,shape=[None,None,300])
Y = tf.placeholder(dtype=tf.int64, shape=[None])
def RNN(X,numunit,name,output_dim):
    with tf.variable_scope(name):
#         layers=[tf.nn.rnn_cell.BasicLSTMCell(n) for n in numunit]
#         lstm_cell =tf.nn.rnn_cell.MultiRNNCell(layers)
        lstm_cell=tf.nn.rnn_cell.BasicLSTMCell(num_units=numunit[-1])
        outputs, states = tf.nn.dynamic_rnn(lstm_cell, X,initial_state=lstm_cell.zero_state(batch_size=tf.shape(X)[0],dtype=tf.float32), dtype=tf.float32)
        index=tf.range(0,tf.shape(X)[0])*tf.shape(X)[1]+(length-1)        
        flat=tf.reshape(outputs,[-1,numunit[-1]])
        relevant=tf.nn.relu(tf.gather(flat,index))
        w=tf.get_variable(name='weight',shape=[numunit[-1],output_dim],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        b=tf.get_variable(name='bias',shape=[output_dim],initializer=tf.constant_initializer(0.1),dtype=tf.float32)
        return(tf.matmul(relevant, w) + b)
#         return relevant

logits=RNN(X,[100,150],'sentence_vector',num_class)
loss=tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=tf.one_hot(Y,num_class))
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 20, 0.96)
optimize=tf.train.AdamOptimizer(learning_rate=starter_learning_rate).minimize(loss)
step = 0
saver = tf.train.Saver(max_to_keep=1)
checkpoint_dir='model'
try:
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    saver.restore(sess, checkpoint)
    files = os.listdir(checkpoint_dir)
    for file in files:
        if 'data' in file:
            if int(file.split('.')[0].split('-')[-1]) > step:
                step = int(file.split('.')[0].split('-')[-1])
    step = step + 1
    print('Load model Complete at step: {}'.format(step))
except Exception:
    print('Load model fail')
    step = 0

/home/sysadmin/virtualenv/lstm/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [5]:
sess.run(tf.global_variables_initializer())

In [12]:
for epoch in range(step, (step + max_epoch) // max_epoch * max_epoch):
    L=0
    num=0
    for i in np.split(len_train,range(batch_size,len(data_train),batch_size)):
        data=data_train[i]
        label=label_train[i]
        le=[len(d) for d in data]
        data=np.array([np.pad(i,[[0,max(le)-len(i)],[0,0]],'constant') for i in data])
        _,l=sess.run([optimize,loss],feed_dict={X:data,Y:label,length:le,global_step:epoch})
        num+=len(l)
        L+=sum(l)
        print('Epoch {}: loss {:.7f} {}'.format(epoch,L/num,data.shape),end='\r')
    print('\n')
    if  ((epoch+1)%10==0):
        true=0
        for i in np.split(len_train,range(batch_size,len(data_train),batch_size)):
            data=data_train[i]
            label=label_train[i]
            le=[len(d) for d in data]
            data=np.array([np.pad(i,[[0,max(le)-len(i)],[0,0]],'constant') for i in data])
            predict=sess.run(tf.nn.softmax(logits),feed_dict={X:data,Y:label,length:le})
            true+=np.sum(np.argmax(predict,axis=1)==label)
        print('in train: ',true/data_train.shape[0])
        true=0
        for i in np.split(len_vaild,range(batch_size,len(data_vaild),batch_size)):
            data=data_vaild[i]
            label=label_vaild[i]
            le=[len(d) for d in data]
            data=np.array([np.pad(i,[[0,max(le)-len(i)],[0,0]],'constant') for i in data])
            predict=sess.run(tf.nn.softmax(logits),feed_dict={X:data,Y:label,length:le})
            true+=np.sum(np.argmax(predict,axis=1)==label)
        print('in vaild: ',true/data_vaild.shape[0])
        saver.save(sess, checkpoint_dir + '/model', global_step=epoch)        

INFO:tensorflow:Restoring parameters from model/model-149
Load model Complete at step: 150
Epoch 150: loss 0.4215158 (33, 2709, 300)

Epoch 151: loss 0.2447007 (33, 2709, 300)

Epoch 152: loss 0.2220982 (33, 2709, 300)

Epoch 153: loss 0.1433384 (33, 2709, 300)

Epoch 154: loss 0.0908898 (33, 2709, 300)

Epoch 155: loss 0.0549598 (33, 2709, 300)

Epoch 156: loss 0.0383015 (33, 2709, 300)

Epoch 157: loss 0.0284213 (33, 2709, 300)

Epoch 158: loss 0.0235470 (33, 2709, 300)

Epoch 159: loss 0.0206097 (33, 2709, 300)

in train:  0.99523261548578
in vaild:  0.7408491947291361
Epoch 160: loss 0.0181543 (33, 2709, 300)

Epoch 161: loss 0.0158842 (33, 2709, 300)

Epoch 162: loss 0.0149060 (33, 2709, 300)

Epoch 163: loss 0.0137766 (33, 2709, 300)

Epoch 164: loss 0.0125735 (33, 2709, 300)

Epoch 165: loss 0.0117956 (33, 2709, 300)

Epoch 166: loss 0.0115199 (33, 2709, 300)

Epoch 167: loss 0.0110112 (33, 2709, 300)

Epoch 168: loss 0.0108830 (33, 2709, 300)

Epoch 169: loss 0.0104916 (33, 270

In [ ]:
length=tf.placeholder(dtype=tf.int32,shape=[None])
global_step = tf.placeholder(tf.int32)
X =tf.placeholder(dtype=tf.float32,shape=[None,None,300])
Y = tf.placeholder(dtype=tf.int64, shape=[None])
def RNN(X,numunit,name,output_dim):
    with tf.variable_scope(name):
        layers=[tf.nn.rnn_cell.BasicLSTMCell(n) for n in numunit]
        lstm_cell =tf.nn.rnn_cell.MultiRNNCell(layers)
        outputs, states = tf.nn.dynamic_rnn(lstm_cell, X,initial_state=lstm_cell.zero_state(batch_size=tf.shape(X)[0],dtype=tf.float32), dtype=tf.float32)
        index=tf.range(0,tf.shape(X)[0])*tf.shape(X)[1]+(length-1)        
        flat=tf.reshape(outputs,[-1,numunit[-1]])
        relevant=tf.nn.tanh(tf.gather(flat,index))
        w=tf.get_variable(name='weight',shape=[numunit[-1],output_dim],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        b=tf.get_variable(name='bias',shape=[output_dim],initializer=tf.constant_initializer(0.1),dtype=tf.float32)
        return(tf.matmul(relevant, w) + b)
#         return relevant

logits=RNN(X,[100,150],'sentence_vector',num_class)
loss=tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=tf.one_hot(Y,num_class))
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 20, 0.96)
optimize=tf.train.AdamOptimizer(learning_rate=starter_learning_rate).minimize(loss)
sess.run(tf.global_variables_initializer())

step = 0
saver = tf.train.Saver()
checkpoint_dir='model-2layer'
try:
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    saver.restore(sess, checkpoint)
    files = os.listdir(checkpoint_dir)
    for file in files:
        if 'data' in file:
            if int(file.split('.')[0].split('-')[-1]) > step:
                step = int(file.split('.')[0].split('-')[-1])
    step = step + 1
    print('Load model Complete at step: {}'.format(step))
except Exception:
    print('Load model fail')
    step = 0

In [4]:

for epoch in range(step, (step + max_epoch) // max_epoch * max_epoch):
    L=0
    num=0
    for i in np.split(len_train,range(batch_size,len(data_train),batch_size)):
        data=data_train[i]
        label=label_train[i]
        le=[len(d) for d in data]
        data=np.array([np.pad(i,[[0,max(le)-len(i)],[0,0]],'constant') for i in data])
        _,l=sess.run([optimize,loss],feed_dict={X:data,Y:label,length:le,global_step:epoch})
        num+=len(l)
        L+=sum(l)
        print('Epoch {}: loss {:.7f} {}'.format(epoch,L/num,data.shape),end='\r')
    print('\n')
    if  ((epoch+1)%10==0):
        true=0
        for i in np.split(len_train,range(batch_size,len(data_train),batch_size)):
            data=data_train[i]
            label=label_train[i]
            le=[len(d) for d in data]
            data=np.array([np.pad(i,[[0,max(le)-len(i)],[0,0]],'constant') for i in data])
            predict=sess.run(tf.nn.softmax(logits),feed_dict={X:data,Y:label,length:le})
            true+=np.sum(np.argmax(predict,axis=1)==label)
        print('in train: ',true/data_train.shape[0])
        true=0
        for i in np.split(len_vaild,range(batch_size,len(data_vaild),batch_size)):
            data=data_vaild[i]
            label=label_vaild[i]
            le=[len(d) for d in data]
            data=np.array([np.pad(i,[[0,max(le)-len(i)],[0,0]],'constant') for i in data])
            predict=sess.run(tf.nn.softmax(logits),feed_dict={X:data,Y:label,length:le})
            true+=np.sum(np.argmax(predict,axis=1)==label)
        print('in vaild: ',true/data_vaild.shape[0])
        saver.save(sess, checkpoint_dir + '/model', global_step=epoch)        

/home/sysadmin/virtualenv/lstm/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from model-2layer/model-29
Load model Complete at step: 30
Epoch 30: loss 0.1542258 (33, 2709, 300)

Epoch 31: loss 0.1363260 (33, 2709, 300)

Epoch 32: loss 0.1292444 (33, 2709, 300)

Epoch 33: loss 0.1009403 (33, 2709, 300)

Epoch 34: loss 0.0883385 (33, 2709, 300)

Epoch 35: loss 0.0902528 (33, 2709, 300)

Epoch 36: loss 0.0690480 (33, 2709, 300)

Epoch 37: loss 0.0604337 (33, 2709, 300)

Epoch 38: loss 0.0690818 (33, 2709, 300)

Epoch 39: loss 0.0388751 (33, 2709, 300)

in train:  0.958737465066579
in vaild:  0.7906295754026355
Epoch 40: loss 0.0317109 (33, 2709, 300)

Epoch 41: loss 0.0277658 (33, 2709, 300)

Epoch 42: loss 0.0289027 (33, 2709, 300)

Epoch 43: loss 0.4864684 (33, 2709, 300)

Epoch 44: loss 0.8571459 (33, 2709, 300)

Epoch 45: loss 0.3733282 (33, 2709, 300)

Epoch 46: loss 0.1956804 (33, 2709, 300)

Epoch 47: loss 0.1251460 (33, 2709, 300)

Epoch 48: loss 0.0796791 (33, 2709, 300)

Epoch 49: loss 0.0597880 (33, 2709, 300)

in tr

Epoch 197: loss 0.0079539 (33, 2709, 300)

Epoch 198: loss 0.0080423 (33, 2709, 300)

Epoch 199: loss 0.0079484 (33, 2709, 300)

in train:  0.9955614006246918
in vaild:  0.8184480234260615


In [ ]:
tmp=dict()
def test(strings):
    for i in glob.glob('data/*.txt'):
        tmp[len(tmp)]=i.split('/')[-1].split('.')[0]
    for i in strings:
        words=re.sub(' +',' ',re.compile('[^\w0-9 ]').sub('',data[j])).split(' ')
        data.append(np.array([word.get_word_vector(j) for j in words]))
    le=[len(d) for d in data]
    data=np.array([np.pad(i,[[0,max(le)-len(i)],[0,0]],'constant') for i in data])
    predict=sess.run(tf.argmax(tf.nn.softmax(logits)),feed_dict={X:data,length:le})
    